<a href="https://colab.research.google.com/github/Sadhowon/clase/blob/main/Practica_Banco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Descargar base de datos


In [10]:
!gdown 1puf9UBuzaVqMW3ZMMXhd626E5yVJd_QX

Downloading...
From: https://drive.google.com/uc?id=1puf9UBuzaVqMW3ZMMXhd626E5yVJd_QX
To: /content/data_berka.db
100% 70.2M/70.2M [00:00<00:00, 151MB/s]


1.1) Cargamos la extension sql(nos permite ejecutar comandos sql directamente en colab)

In [21]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


1.2) Conectar a la Base de datos

In [8]:
%sql sqlite:///data_berka.db

1.3) Comprobamos el esquema de la BBDD
Para ejecutar comandos sql, se debe añadir "%sql comando" o en multiples lineas "%%sql
"comando_linea1
comando_linea2"

In [23]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type = "table"

 * sqlite:///data_berka.db
Done.


name,type
DISTRICT,table
ACCOUNT,table
TRANS,table
CLIENT,table
DISPOSITION,table
CREDIT_CARD,table
LOAN,table
PERMANENT_ORDER,table


# 2) Ejercicicos

2.1) Muestre los datos de la tabla cliente

In [24]:
%sql SELECT * FROM CLIENT LIMIT 5

 * sqlite:///data_berka.db
Done.


client_id,birth_number,district_id
1,706213,18
2,450204,1
3,406009,1
4,561201,5
5,605703,5


2.2) Liste todos los clientes con cuentas activas y prestamos, inluyendo dinero prestado y saldos de cuenta

In [26]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type = "table" #account_id, loan_id y amount

 * sqlite:///data_berka.db
Done.


name,type
DISTRICT,table
ACCOUNT,table
TRANS,table
CLIENT,table
DISPOSITION,table
CREDIT_CARD,table
LOAN,table
PERMANENT_ORDER,table


In [50]:
%%sql
select c.client_id as cliente,
ROUND(SUM(l.amount), 2) as importe_prestado,
ROUND(SUM(CASE
  WHEN t.type = 'PRIJEM' THEN t.amount
  WHEN t.type = 'VYDAJ' then -t.amount
  else 0
  end), 2) as balance_cuenta
from CLIENT c
left join  disposition d on d.client_id = c.client_id
left join ACCOUNT a on a.account_id = d.account_id
left join loan l on l.account_id = a.account_id
left join trans t on t.account_id = a.account_id
where d.type = 'OWNER'
group by c.client_id
limit 5

 * sqlite:///data_berka.db
Done.


cliente,importe_prestado,balance_cuenta
1,None,13466.5
2,38695056.0,260069.7
4,None,51096.2
6,None,33720.6
7,None,28088.6


In [58]:
%%sql
select c.client_id as cliente, COUNT(a.account_id) as total_cuentas
from client c
left join disposition d on d.client_id = c.client_id
left join ACCOUNT a on a.account_id = d.account_id
group by c.client_id
having total_cuentas > 1

 * sqlite:///data_berka.db
Done.


cliente,total_cuentas


In [87]:
%%sql
SELECT trans_id, date, amount
FROM trans
where date like '98%'
ORDER BY amount DESC
LIMIT 10;

 * sqlite:///data_berka.db
Done.


trans_id,date,amount
291346,981215,87400.0
1306770,980606,87300.0
1207200,980619,85800.0
1170721,980615,81900.0
706371,980610,80100.0
141211,980625,78300.0
1155373,980618,78000.0
623347,980612,77400.0
768915,980620,77100.0
816974,980621,76800.0


In [96]:
%%sql
SELECT
  CASE
    WHEN t.account_id IN (SELECT account_id FROM loan) THEN 'Con préstamo'
    ELSE 'Sin préstamo'
  END AS tipo_cuenta,
  round(AVG(balance), 2) AS saldo_promedio
FROM trans t
GROUP BY tipo_cuenta;


 * sqlite:///data_berka.db
Done.


tipo_cuenta,saldo_promedio
Con préstamo,45467.14
Sin préstamo,36979.08


In [112]:
%%sql
select c.client_id as cliente
from client c
left join disposition d on d.client_id = c.client_id
left join ACCOUNT a on a.account_id = d.account_id
left join trans t on t.account_id = a.account_id
left join loan l on l.account_id = a.account_id
where t.date like '98%' and l.account_id is null
group by c.client_id
having COUNT(t.trans_id) > 10
limit 10

 * sqlite:///data_berka.db
Done.


cliente
1
4
5
6
7
8
9
10
11
12


In [132]:
%%sql
WITH Limits AS (
    SELECT MAX(TRANS.balance) * 0.5 AS medium_limit FROM TRANS
),
latest_trans AS (
    SELECT account_id, MAX(trans_id) AS max_trans_id
    FROM TRANS
    GROUP BY account_id
),
client_balances AS (
    SELECT
        CLIENT.client_id,
        TRANS.account_id,
        TRANS.balance AS LAST_BALANCE,
        CASE
            WHEN TRANS.balance <= 0 THEN 'BAJO'
            WHEN TRANS.balance > 0 AND TRANS.balance <= (SELECT medium_limit FROM Limits) THEN 'MEDIO'
            WHEN TRANS.balance > (SELECT medium_limit FROM Limits) THEN 'ALTO'
            ELSE 'NAN'
        END AS CALIDAD_CLIENTE,
        LOAN.amount AS importe_prestado
    FROM CLIENT
    JOIN DISPOSITION ON CLIENT.client_id = DISPOSITION.client_id
    JOIN TRANS ON DISPOSITION.account_id = TRANS.account_id
    JOIN LOAN ON TRANS.account_id = LOAN.account_id
    JOIN latest_trans ON TRANS.account_id = latest_trans.account_id AND TRANS.trans_id = latest_trans.max_trans_id
)
SELECT
    CALIDAD_CLIENTE,
    COUNT(DISTINCT client_id) AS num_clientes,
    SUM(importe_prestado) AS total_importe_prestado,
    ROUND(COUNT(DISTINCT client_id) * 100.0 / SUM(COUNT(DISTINCT client_id)) OVER (), 2) AS proporcion_clientes,
    ROUND(SUM(importe_prestado) * 100.0 / SUM(SUM(importe_prestado)) OVER (), 2) AS proporcion_importe_prestado
FROM client_balances
GROUP BY CALIDAD_CLIENTE
ORDER BY CALIDAD_CLIENTE;

 * sqlite:///data_berka.db
(sqlite3.OperationalError) misuse of aggregate: max()
[SQL: with limits as (select max(t.balance)/2 as medium_limit from trans)
select 
  c.client_id,
  t.trans_id,
  t.balance as last_balance,
  case
    when max(t.balance) > 0 then 'Bajo'
    when max(t.balance) > 0 and max(t.balance) >= (select medium_limit from limits) then 'Medio'
    when max(t.balance) > (select medium_limit from limits) then 'Alto'
    else 'NAN'
  end as calidad_cliente
from client c
join disposition d on d.client_id = c.client_id
join trans t on t.account_id = d.account_id
join (
    select account_id, max(trans_id) as max_trans_id
    from trans
    group by account_id
) latest_trans on t.account_id = latest_trans.account_id and t.trans_id = latest_trans.max_trans_id
group by c.client_id
limit 10]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
